### Importing Libraries

In [7]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import cupy as cp  # CuPy for GPU-based NumPy operations
import numpy as np
import tensorflow as tf
import scipy
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Feature Extraction

**Define the paths for the images**

In [2]:
# Define paths
dataset_dir = "textures 3"
categories = ['cotton', 'corduroy', 'denim', 'linin', 'wool']

**Canny Edge Detection**

In [3]:
def extract_canny_edge_detection(image):
    """ image must be passes to the function in grayscale"""
    # Step 1: Enhance contrast (optional)
    equalized_image = cp.asarray(cv2.equalizeHist(cp.asnumpy(image)))
    
    # Step 2: Apply Gaussian Blur to reduce noise
    blurred_image = cp.asarray(cv2.GaussianBlur(cp.asnumpy(equalized_image), (3, 3), 1))
    
    # Step 3: Apply Canny Edge Detection with adjusted thresholds (convert back and forth)
    edges = cp.asarray(cv2.Canny(cp.asnumpy(blurred_image), 30, 30))

    return edges

**Gabor Filtering**

In [4]:
def extract_gabor_filters(image):
    """ image must be in grayscale"""
    
    def build_kernels():
        # Parameters
        gabor_kernels = []
        angles = [0, cp.pi/4, cp.pi/2, 3*cp.pi/4]  # Use CuPy for angles
        ksize = 31  # Size of the filter
        sigma = 4.0  # Standard deviation of the Gaussian envelope
        lambd = 10.0  # Wavelength of the sinusoidal factor
        gamma = 0.5  # Spatial aspect ratio
        psi = 0  # Phase offset

        # Create Gabor kernels
        for theta in np.deg2rad([45, 135]):  # Convert degrees to radians
            kernel = cp.asarray(cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)) # Using Cupy array
            gabor_kernels.append(kernel)

        return gabor_kernels


    gabor_kernels = build_kernels()
    
    gabor_features = []

    for kernel in gabor_kernels:
        fimg = cp.asarray(cv2.filter2D(cp.asnumpy(image), cv2.CV_8UC3, cp.asnumpy(kernel)))
        gabor_features.append(fimg)

    gabor_features = cp.array(gabor_features).flatten()

    return gabor_features

**Local Binary Pattern**

In [5]:
def extract_local_binary_pattern(image):

    # Parameters 
    radius = 1
    n_points = 8 * radius

    
    lbp = local_binary_pattern(cp.asnumpy(image), n_points, radius, method="uniform")
    (hist, _) = cp.histogram(cp.asarray(lbp).ravel(), bins=cp.arange(0, n_points + 3),
                             range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)

    return hist

**Feature Extraction**

In [6]:
# Function to extract features from an image
def extract_features(image):
    # Convert to grayscale using CuPy arrays
    gray = cp.asarray(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

    # Canny edge detection
    edges = extract_canny_edge_detection(gray)
    
    # Gabor Filter responses
    gabor_features=  extract_gabor_filters(gray)
    
    # Local Binary Patterns (LBP)
    hist = extract_local_binary_pattern(gray)
    
    # Combine features: edges, Gabor, and LBP
    features = cp.hstack([edges.flatten(), gabor_features, hist])
    features = cp.asnumpy(features)
    
    return features # Return features back as NumPy array for further processing

**Image Augmentation**

In [ ]:
# Prepare dataset and labels
data = []
labels = []

# Image Augmentation using TensorFlow
datagen = ImageDataGenerator(
    #rotation_range=15,        # Random rotations up to 15 degrees
    #width_shift_range=0.1,    # Horizontal shifts
    #height_shift_range=0.1,   # Vertical shifts
    horizontal_flip=True,     # Flip images horizontally
    vertical_flip=True,       # Flip images vertically
    zoom_range=0.2,           # Random zoom
    brightness_range=[0.8, 1.2], # Brightness adjustment
    shear_range=0.1           # Shear transformation
)

for category in categories:
    path = os.path.join(dataset_dir, category)
    label = category
    
    for count,img_name in enumerate(os.listdir(path),start=1):

        if count==30:
            break
        
        img_path = os.path.join(path, img_name)
        image = cv2.imread(img_path)
        
        try:
            # Apply data augmentation and extract features
            image = cv2.resize(image, (128, 128))  # Resize to a fixed size
            image = np.expand_dims(image, axis=0)  # Prepare for augmentation
            aug_iter = datagen.flow(image, batch_size=1)

            # Perform 5 augmentations per image
            for _ in range(5):
                aug_img = next(aug_iter)[0].astype(np.uint8)

                plt.figure(figsize=(8, 4))

                #Original image
                plt.subplot(1, 2, 1)
                plt.imshow(image)
                plt.title("Original Image")
                
                
                # Augmented image
                plt.subplot(1, 2, 2)
                plt.imshow(aug_img)
                plt.title("Augmented Image")


        except Exception as e:
            print(img_path,img_name)

            
        

**Data** contains the input data (X) <br>
**Labels** contains the output data (Y) 

**Saving the data and labels**

In [13]:
# Convert lists to NumPy arrays
data_np = np.array(data)
labels_np = np.array(labels)

if not os.path.exists("Extracted_features"):
    os.makedirs("Extracted_features")
# Save to a .npz file
np.savez('Extracted_features\\data_labels.npz', data=data_np, labels=labels_np)


**Loading the saved data and labels**

In [4]:
# Load from the .npz file
loaded = np.load('Extracted_features\\data_labels.npz')
data_loaded = loaded['data']
labels_loaded = loaded['labels']

In [ ]:
# Checking the shape of the dataset and the labels
print(f"Dataset shape: {data_loaded.shape}")
print(f"Labels shape: {labels_loaded.shape}")

From the above output we can see that we have **49162 features** and **17780 rows**

In [6]:
data = data_loaded
labels = labels_loaded

### Label Encoding of target variable

In [7]:
# Encode labels
le = LabelEncoder()
labels = le.fit_transform(labels)

### Train-test split

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(np.array(data), np.array(labels), test_size=0.3, random_state=42)
print("X_train shape: ",X_train.shape)
print("X_test shape: ",X_test.shape)
print("y_train shape: ",y_train.shape)
print("y_test shape: ",y_test.shape)

### Standardization

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Dimensionality Reduction using PCA(Principal Component Analysis)

In [11]:
# Dimensionality Reduction using PCA 
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

### Model Training

In [ ]:
# Train a Random Forest Classifier 
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_clf.fit(X_train, y_train)

In [ ]:
# Apply SVM model (using RBF kernel )
svm_clf = SVC(kernel='rbf', random_state=42)
svm_clf.fit(X_train, y_train)

### Model Evaluation

In [ ]:
# Predictions and Evaluation
y_pred = random_forest_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))
print(confusion_matrix(y_test, y_pred))


In [ ]:
# Predictions and Evaluation
y_pred = svm_clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))
print(confusion_matrix(y_test, y_pred))

**Checking class imbalance**

In [ ]:
print(pd.Series(labels_loaded).value_counts())


In [ ]:
print(len(pd.Series(labels_loaded)))

In [ ]:
print((pd.Series(labels_loaded).value_counts() / len(pd.Series(labels_loaded)))*100)


Classes are balanced